In [1]:
%matplotlib auto
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import math
from mpl_toolkits.mplot3d import Axes3D

Using matplotlib backend: <object object at 0x000001A248E4CB60>


### NTU60的动作

In [2]:
action_names = ['drink water 1', 'eat meal/snack 2', 'brushing teeth 3', 'brushing hair 4', 'drop 5', 'pickup 6',
            'throw 7', 'sitting down 8', 'standing up 9', 'clapping 10', 'reading 11', 'writing 12',
            'tear up paper 13', 'wear jacket 14', 'take off jacket 15', 'wear a shoe 16', 'take off a shoe 17',
            'wear on glasses 18','take off glasses 19', 'put on a hat/cap 20', 'take off a hat/cap 21', 'cheer up 22',
            'hand waving 23', 'kicking something 24', 'put/take out sth 25', 'hopping 26', 'jump up 27',
            'make a phone call 28', 'playing with a phone 29', 'typing on a keyboard 30',
            'pointing to sth with finger 31', 'taking a selfie 32', 'check time (from watch) 33',
            'rub two hands together 34', 'nod head/bow 35', 'shake head 36', 'wipe face 37', 'salute 38',
            'put the palms together 39', 'cross hands in front 40', 'sneeze/cough 41', 'staggering 42', 'falling 43',
            'touch head 44', 'touch chest 45', 'touch back 46', 'touch neck 47', 'nausea or vomiting condition 48',
            'use a fan 49', 'punching 50', 'kicking other person 51', 'pushing other person 52',
            'pat on back of other person 53', 'point finger at the other person 54', 'hugging other person 55',
            'giving sth to other person 56', 'touch other person pocket 57', 'handshaking 58',
            'walking towards each other 59', 'walking apart from each other 60']
max_frame = 300
max_body = 2
max_joint = 25

### 读取数据

#### 1.读取skeleton文件

In [3]:
def read_skeleton(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []
            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)
    return skeleton_sequence

#### 2.读取60个样本，每个样本归属于一类动作，共60类

In [4]:
data_file = "../data/ntu60/c003"
data = np.zeros((60, 3,max_frame, max_joint, max_body))   # 60个不同动作的样本坐标
frame_nums = np.zeros((60))
for file in os.listdir(data_file):
    action_class = int(file[file.find('A')+1:file.find('A')+4])   # 动作编号
    action_name = action_names[action_class-1]    # 动作名称
    file_path = os.path.join(data_file, file)    # skeleton文件路径
    skeleton_info = read_skeleton(file_path)   # 读取一个skeleton文件信息
    frame_nums[action_class-1] = skeleton_info['numFrame']
    for n, f in enumerate(skeleton_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < max_joint:
                    data[action_class-1, :, n, j, m] = [v['x'], v['y'], v['z']]
                else:
                    pass

In [5]:
print(data.shape)
print(frame_nums)

(60, 3, 300, 25, 2)
[ 98.  93. 121. 130.  60.  83.  59.  70.  59.  64. 105. 118.  84. 128.
 186. 125.  99. 161.  84.  88.  85.  81.  59.  66.  90.  99.  60. 106.
 114. 142.  52.  84.  71.  77.  80.  71. 105.  87.  73.  53.  74. 101.
  76.  80.  84.  98.  82. 128. 106.  59.  68.  63.  88.  62.  55.  68.
  91.  71.  87.  76.]


#### 3.归一化骨骼

In [6]:
def normal_skeleton(data, frame_num):
    #  use as center joint
    data = np.transpose(data,(3,1,2,0))
    data = data[:,0:frame_num,:,:]
    center_joint = data[0, :, 0, :]
    
    center_jointx = np.mean(center_joint[:,0])
    center_jointy = np.mean(center_joint[:,1])
    center_jointz = np.mean(center_joint[:,2])
    center = np.array([center_jointx, center_jointy, center_jointz])
    data = data - center
    data = np.transpose(data,(3,1,2,0))
    return data

#### 4.读取多帧骨骼数据

In [7]:
k_index = 0
t_index = 50
skeleton_data = data[k_index]
skeleton_data = normal_skeleton(skeleton_data, int(frame_nums[k_index]))

#### 5.绘制单帧二维平面图

In [8]:
np.random.seed(202296)
# 骨骼连接关系
connecting_joint = np.array([2,1,21,3,21,5,6,7,21,9,10,11,1,13,14,15,1,17,18,19,2,23,8,25,12])
# 绘制画布
plt.figure()

# 绘制第t_index帧里的第一个人
skeleton_data_xyz = skeleton_data[:,t_index,:,0]
# skeleton_data_xyz = occ_skeleton_data[:,t_index,:,0]
x = skeleton_data_xyz[0]
y = skeleton_data_xyz[1]
z = skeleton_data_xyz[2]

for v in range(25):
    col = (np.random.random(), np.random.random(), np.random.random())
    k = connecting_joint[v]-1
    plt.plot([x[v],x[k]],[y[v],y[k]], color=col, marker='o')   # 旋转前

# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_zticks([])

# plt.savefig("pic/rotationOccOff.svg",dpi=300,block=True)
plt.show(block=True)

#### 6.构建多帧三维数据

In [8]:
k_index = 0
t_start_index = 5   # 起始时间序号
temporal_size = 10   # 时序长度
time_step = 5
skeleton_data = data[k_index]
skeleton_data = normal_skeleton(skeleton_data, int(frame_nums[k_index]))
# 构建需要画图的数据
temporal_data = np.zeros((3,temporal_size,25,2))
# 需要丢弃的帧序号
miss_index = [2,3,4,8]
for t in range(temporal_size):
    temporal_data[:,t,:,:] = skeleton_data[:,t_start_index+t*time_step,:,:]
    temporal_data[2,t,:,:] = np.zeros((25,2))+t
    # 判断是否为需要丢弃的帧
    if t in miss_index:
        temporal_data[0,t,:,:] = np.zeros((25,2))
        temporal_data[1,t,:,:] = np.zeros((25,2))
    # 将每帧xy中的y坐标向上提取0.1
    temporal_data[1,t,:,:] = temporal_data[1,t,:,:] + 0.05*t

#### 7.绘制三维时序骨骼图

In [9]:
# np.random.seed(202296)
# 骨骼连接关系
connecting_joint = np.array([2,1,21,3,21,5,6,7,21,9,10,11,1,13,14,15,1,17,18,19,2,23,8,25,12])
# 绘制画布
fig = plt.figure()
ax = Axes3D(fig)
ax.view_init(10, -115)
# 绘制3d骨骼点的每一帧
plt.cla()   # 清除当前画布的axes
# ax.set_xlim3d([0,5])
# ax.set_ylim3d([-1.2,1.8])
# ax.set_zlim3d([-0.7,1.2])


# 统计所有帧里最大最小坐标进行画框
boundx_min = []
boundx_max = []
boundy_min = []
boundy_max = []


# 绘制第t_index帧里的第一个人
for t_index in range(temporal_size):
    skeleton_data_xyt = temporal_data[:,t_index,:,0]
    # skeleton_data_xyz = occ_skeleton_data[:,t_index,:,0]
    x = skeleton_data_xyt[0]
    y = skeleton_data_xyt[1]
    t = skeleton_data_xyt[2]
    
    # 绘制时间遮挡矩形框
    boundx_min.append(x.min()-0.1)
    boundx_max.append(x.max()+0.1)
    boundy_min.append(y.min()-0.1-t_index*0.05)
    boundy_max.append(y.max()+0.1-t_index*0.05)

    np.random.seed(202296)
    for v in range(25):
        col = (np.random.random(), np.random.random(), np.random.random())
        k = connecting_joint[v]-1
        ax.plot([t[v],t[k]],[x[v],x[k]],[y[v],y[k]], color=col, marker='o')   # 旋转前
        
# 绘制矩形边界框
x_min = np.array(boundx_min).min()
x_max = np.array(boundx_max).max()
y_min = np.array(boundy_min).min()
y_max = np.array(boundy_max).max()
# for t_index in [2,3,4,8]:
#     ax.plot([t_index,t_index],[x_min,x_max],[y_max+ 0.05*t_index,y_max+ 0.05*t_index],color='silver')
#     ax.plot([t_index,t_index],[x_max,x_max],[y_max+ 0.05*t_index,y_min+ 0.05*t_index],color='silver')
#     ax.plot([t_index,t_index],[x_max,x_min],[y_min+ 0.05*t_index,y_min+ 0.05*t_index],color='silver')
#     ax.plot([t_index,t_index],[x_min,x_min],[y_min+ 0.05*t_index,y_max+ 0.05*t_index],color='silver')
    
# 绘制立方体cube
miss_frame_list = [[2,3,4],[8]]
for cube_idx in range(len(miss_frame_list)):
    cubex_start = x_min
    cubex_end = x_max
    cubey_start = y_min+0.05*miss_frame_list[cube_idx][-1]
    cubey_end = y_max+0.05*miss_frame_list[cube_idx][0]
    for i in miss_frame_list[cube_idx]:
        cubet_start = i-0.5
        cubet_end = i+0.5
        # 开始绘图
        cubex = np.arange(cubex_start, cubex_end, 0.05)
        cubey = np.arange(cubey_start, cubey_end, 0.05)
        cubet = np.arange(cubet_start, cubet_end, 0.05)
        
        cubex, cubey = np.meshgrid(cubex, cubey)
        ax.plot_surface(np.full_like(cubex, cubet_start),cubex, cubey,color="lime",alpha=0.2)
        ax.plot_surface(np.full_like(cubex, cubet_end),cubex, cubey,color="lime",alpha=0.2)
        
        cubet, cubey = np.meshgrid(cubet, cubey)
        ax.plot_surface(cubet,np.full_like(cubet, cubex_start), cubey,color="lime",alpha=0.2)
        ax.plot_surface(cubet,np.full_like(cubet, cubex_end), cubey,color="lime",alpha=0.2)
        
        cubet, cubex = np.meshgrid(cubet, cubex)
        ax.plot_surface(cubet,cubex,np.full_like(cubet, cubey_start),color="lime",alpha=0.2)
        ax.plot_surface(cubet,cubex,np.full_like(cubet, cubey_end),color="lime",alpha=0.2)
    
    

ax.set_xlabel('T')
ax.set_ylabel('X')
ax.set_zlabel('Y')
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.axis('off')
# plt.savefig("pic/temporal_occ_box.svg",dpi=300,block=True)
plt.show(block=True)

C:\Users\admin\AppData\Local\Temp/ipykernel_968/250630379.py:6: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = Axes3D(fig)
